# Capptu Image Aesthetics 

## Toy Triplet Model by XVR

In [1]:
import numpy as np
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, GlobalMaxPooling2D, Dropout, Lambda, Merge, concatenate
from keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow as tf
############## Settings ###########################
batch_size = 8
image_size = 224

embedding_dim = 1000
############## DATA ###########################
def GetRandomImage():
    return np.random.randint(low=0, high=256, size=[image_size,image_size,3])
    
def GetTriplet():
    a = GetRandomImage()
    b = GetRandomImage()
    c = GetRandomImage()
    return a,b,c

def Generate():
    while True:
        list_a = []
        list_b = []
        list_c = []

        for i in range(batch_size):
            a, b, c = GetTriplet()
            list_a.append(a)
            list_b.append(b)
            list_c.append(c)
            
        A = preprocess_input(np.array(list_a, dtype='float32'))
        B = preprocess_input(np.array(list_b, dtype='float32'))
        C = preprocess_input(np.array(list_c, dtype='float32'))
        label = np.ones((batch_size,2))
        yield [A, B, C], label


train_generator = Generate()
test_generator = Generate()
batch = next(train_generator)
       
############## LOSS ########################### 
def identity_loss(y_true, y_pred):
    
    r = y_true[0] - y_pred[0]
    
    return K.mean(y_pred - 0 * y_true)

def fake_loss(X):
    item1, item2, item3 = X
    # item = X
    ##loss = K.sum(A * item1, axis=-1, keepdims=True)
    loss = K.sum(K.square(item1-item2),axis=-1,keepdims=True)
    
    return loss


def Le(X):
    a, p, n = X
    m = 0.2
    loss = K.relu(m + K.sum(K.square(a-p),axis=-1,keepdims=True) - K.sum(K.square(a-n),axis=-1,keepdims=True))
    return loss
def Ld_1(X):
    a, p, n = X
    m = 0.2
    loss = K.relu(m+ K.sqrt(K.sum(K.square(a),axis=-1,keepdims=True)) - K.sqrt(K.sum(K.square(n),axis=-1,keepdims=True))) 
    return loss

def triplet_loss(y_true,y_pred):
    sa = y_true[0]
    sp = y_true[1]
    sn = y_true[2]
    
    ld = y_pred[0]
    le = y_pred[0]
    
    return (sn - sa)*ld + le

############## Model ########################### 
def GetBaseModel():
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    dense_1 = Dense(embedding_dim)(x)
    normalized = Lambda(lambda  x: K.l2_normalize(x,axis=1))(dense_1)
    base_model = Model(base_model.input, normalized, name="base_model")
    return base_model

def GetMyModel(base_model):
    input_1 = Input((image_size,image_size,3))
    input_2 = Input((image_size,image_size,3))
    input_3 = Input((image_size,image_size,3))

    r1 = base_model(input_1)
    r2 = base_model(input_2)
    r3= base_model(input_3)

    loss_le = Lambda(Le)([r1,r2,r3])
    loss_ld1 = Lambda(Ld_1)([r1,r2,r3])
    loss = concatenate([loss_le,loss_ld1],axis=-1)
    
    ##loss_ld1 = Lambda(Ld_1)
    
    model = Model(inputs=[input_1, input_2, input_3], outputs=loss)
    model.compile(loss=identity_loss, optimizer=Adam(0.000003))
    return model

base_model = GetBaseModel()
model = GetMyModel(base_model)
model.summary()

#print(model.predict_on_batch(batch[0]))
print(model.predict_on_batch(batch[0]))

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
base_model (Model)              (None, 1000)         25636712    input_2[0][0]                    
                                                                 input_3[0][0]                    
          

In [2]:
model.fit_generator(train_generator, 
                    validation_data=test_generator, 
                    epochs=5, 
                    verbose=1, 
                    workers=4,
                    steps_per_epoch=100, 
                    validation_steps=30)

Epoch 1/5
100/100 [==============================] - 44s 441ms/step - loss: 0.1986 - val_loss: 0.2000
Epoch 2/5
100/100 [==============================] - 31s 313ms/step - loss: 0.2016 - val_loss: 0.2000
Epoch 3/5
100/100 [==============================] - 32s 315ms/step - loss: 0.1982 - val_loss: 0.2000
Epoch 4/5
100/100 [==============================] - 32s 322ms/step - loss: 0.2014 - val_loss: 0.2000
Epoch 5/5
100/100 [==============================] - 32s 319ms/step - loss: 0.2016 - val_loss: 0.2000


## Triplet Loss Model Capptu 

In [37]:
import numpy as np
import tensorflow as tf
import pandas as pd
import h5py
import random
import matplotlib.pyplot as plt
import cv2
import time 
import keras.backend as K
import keras 
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.utils import plot_model
from keras.optimizers import Adam, SGD
from keras.losses import mean_absolute_error, categorical_crossentropy,mean_absolute_error
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Lambda, concatenate
from keras.models import Input,Model



### Global Parameters

In [39]:
m_e = 0.2
m_d = 0.3

TRAINING_SIZE = 100000
TESTING_SIZE = 2000
VALIDATION_SIZE = 2000

TOTAL_IMAGES = 180000
IMAGE_SIZE = 224

BATCH_SIZE = 8
TRIPLET_INDEX = 0

ENCODINGS_DIM = 1000






### Loading Dataset

In [3]:
arod = h5py.File('./AROD_HDF/AROD.hdf','r')
triplets = pd.read_csv('./triplets.csv').get_values()[0:TRAINING_SIZE]
training_set = triplets[:,1:4]

In [4]:
def get_triplet():
    global TRIPLET_INDEX
    triplet = training_set[TRIPLET_INDEX]
    
    a = arod['IMAGES'][triplet[0]]
    p = arod['IMAGES'][triplet[1]]
    n = arod['IMAGES'][triplet[2]]
    
    sa = arod['SCORES'][triplet[0]][0]        
    sp = arod['SCORES'][triplet[1]][0]        
    sn = arod['SCORES'][triplet[2]][0]        
    TRIPLET_INDEX = TRIPLET_INDEX + 1
    return a, p, n, sa, sp, sn 

In [5]:
a, p, n, sa, sp, sn = get_triplet()

In [26]:
def Generate():
    while True:
        list_a = []
        list_p = []
        list_n = []
        label = []

        for i in range(BATCH_SIZE):
            a, p, n, sa, sp, sn = get_triplet()
            list_a.append(a)
            list_p.append(p)
            list_n.append(n)
            label.append([sa,sn])
            
        A = preprocess_input(np.array(list_a, dtype = 'float32'))
        B = preprocess_input(np.array(list_p, dtype = 'float32'))
        C = preprocess_input(np.array(list_n, dtype = 'float32'))
        label = np.array(label,dtype = 'float32')
        yield [A, B, C], label
        #return [A,B,C], label

### Loss Function





$L_e (a,p,n) = [m_e + |\Phi_a - \Phi_p|^2  - |\Phi_a - \Phi_n|^2 ]$ 

$L_d (a,p,n) = sign (s(n) - s(a) )  [m_d + |\Phi_a - \Phi_n| ]  $ 

$Loss = L_d + L_e$

Where :

$\Phi_i:$ Encodings of $ith$ Image

$m_d, m_e:$ Margins to avoid Trivial loss response





In [29]:
train_generator = Generate()
batch = next(train_generator)
############## LOSS Function ########################### 
def identity_loss(y_true, y_pred):
    r = y_true[0] - y_pred[0]
    return K.mean(y_pred - 0 * y_true)


def Le(X):
    a, p, n = X
    m = 0.2
    loss = K.relu(m + K.sum(K.square(a-p),axis=-1,keepdims=True) - K.sum(K.square(a-n),axis=-1,keepdims=True))
    return loss
def Ld_1(X):
    a, p, n = X
    m = 0.2
    loss = K.relu(m+ K.sqrt(K.sum(K.square(a),axis=-1,keepdims=True)) - K.sqrt(K.sum(K.square(n),axis=-1,keepdims=True)))
    return loss

def triplet_loss(y_true,y_pred):
    sa = y_true[0]
    sp = y_true[1]
    sn = y_true[2]
    
    ld = y_pred[0]
    le = y_pred[0]
    
    return (sn - sa)*ld + le



### Neural Net model 
<img src="./Capptu model Resnet50_tripletloss.png">






### Summary






In [40]:
############## Model ########################### 
def GetBaseModel():
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.5)(x)
    dense_1 = Dense(ENCODINGS_DIM)(x)
    normalized = Lambda(lambda  x: K.l2_normalize(x,axis=1))(dense_1)
    base_model = Model(base_model.input, normalized, name="base_model")
    return base_model

def GetModel(base_model):
    input_1 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_2 = Input((IMAGE_SIZE,IMAGE_SIZE,3))
    input_3 = Input((IMAGE_SIZE,IMAGE_SIZE,3))

    r1 = base_model(input_1)
    r2 = base_model(input_2)
    r3= base_model(input_3)

    loss_le = Lambda(Le)([r1,r2,r3])
    loss_ld1 = Lambda(Ld_1)([r1,r2,r3])
    loss = concatenate([loss_le,loss_ld1],axis=-1)
    
    
    model = Model(inputs=[input_1, input_2, input_3], outputs=loss)
    model.compile(loss=identity_loss, optimizer=Adam(0.000003))
    return model
base_model = GetBaseModel()
model = GetModel(base_model)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
base_model (Model)              (None, 1000)         25636712    input_7[0][0]                    
                                                                 input_8[0][0]                    
          

### Training Model

In [ ]:
model.fit_generator(train_generator, 
                    epochs=5, 
                    verbose=1, 
                    workers=4,
                    steps_per_epoch=100, 
                    validation_steps=30)

Epoch 1/5
100/100 [==============================] - 52s 522ms/step - loss: 0.2027
Epoch 2/5
100/100 [==============================] - 36s 362ms/step - loss: 0.2007
Epoch 3/5
100/100 [==============================] - 36s 362ms/step - loss: 0.2024
Epoch 4/5
100/100 [==============================] - 37s 366ms/step - loss: 0.1980
Epoch 5/5
  7/100 [=>............................] - ETA: 32s - loss: 0.2008